In [28]:
import glob
from os.path import join, basename, splitext

In [29]:
def readFrameAnnotation(annotationFile):
    """
        read annotation file
        return the list of annotation ([start, end], gesture)
    """
    anno = []
    for l in open(annotationFile).read().splitlines():
        s = l.split(' ')
        anno += [ ([int(s[1]), int(s[2])], int(s[0])-1)]
    return anno

In [70]:
class Annotation:
    def __init__(self, annotFile=None):
        self.seqs = []
        self.gesture = []
        
        if not annotFile is None:
            for l in open(annotFile).read().splitlines():
                s = l.split(' ')
                self.seqs.append( (int(s[1]), int(s[2])) )
                self.gesture.append(int(s[0])-1)
    
    def addAnnotation(self, annotFile):
        for l in open(annotFile).read().splitlines():
            s = l.split(' ')
            self.seqs.append(int(s[1]), int(s[2]))
            self.gesture.append(int(s[0])-1)
            
    def __len__(self):
        return len(self.gesture)
    
    def __getitem__(self, i):
        return self.seqs[i], self.gesture[i]
    
    def __str__(self):
        s = "Annotation("
        if len(self) < 5 :
            for i in range(len(self)):
                s += str(self[i])
        else :
            s += str(self[0]) + ' ... ' + str(self[-1])
        return s +")"
    
    def __repr__(self):
        s = "Annotation("
        for i in range(len(self)):
            s += str(self[i])
        return s +")"

In [76]:
class VideoDataset:
    def __init__(self, folderName):
        """
            Look for all .mp4 files in the folder
            The folder must have a annotation folder inside

            return a list of [ (video path, annotation) ]
        """
        p = join(folderName, "*.mp4")
        annoPath = join(folderName, "annotation")
        
        self.annotations = []
        self.videoList = []
        self.folderName = folderName
        
        for v in glob.glob(p):
            bname = basename(v)
            annoFile = join(annoPath, splitext(bname)[0])
            anno = Annotation(annoFile)
            self.annotations.append(anno)
            self.videoList.append(bname)
            
    def __len__(self):
        return len(self.videoList)
    
    def __getitem__(self, i):
        return self.videoList[i], self.annotations[i]
    
    def __repr__(self):
        s = "VideoDataset("
        for i in range(len(self)):
            s += repr(self[i]) + '\n'
        return s + ')'
    
    def __str__(self):
        s = "VideoDataset("
        if len(self) < 5 :
            for i in range(len(self)):
                s += str(self[i]) + '\n'
        else :
            s += str(self[0]) + '\n'*2 + str(self[1]) 
            s += '\n'*2 + ' ... ' + '\n'*2
            s += str(self[-1])
        return s + ')'
    
    def addFolder(self, folder):
        p = join(folderName, "*.mp4")
        for v in glob.glob(p):
            bname = basename(v)
            if not bname in self.videoList:
                annoFile = join(annoPath, splitext(bname)[0])
                annotation = Annotation(annoFile)
                self.annotations.append(annotation)
                self.videoList.append(bname)